In [1]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path: sys.path.append("..")

# Segmentation and boundary-box regression

In [2]:
from src.utils.preprocess import tensor2numpy, image2axial,mask_threshold
from src.utils.utils import split_custom_filename
from src.segmentation import cam
from src.classifier.agent import load_trainer
from src.segmentation.cam import CAMS
from src.segmentation import segmentation
import nibabel as nib


import numpy as np
import torchvision
import torch
import matplotlib.pyplot as plt

## Train on ROI

In [3]:
from src.classifier import agent

roi_trainer,roi_dataset, roi_model =  agent.load_trainer('roi2');

# Ideas: Create a new dataloader which preprocess data for roi?
# * Basically insert ROIAlign in dataloading. Bad point: Not saved on disk
# * Save boundaries in some file to be loaded. When loading file run ROIAlign on them
# Boundaries needed for AD, MCI and CN
# 

Architecture [ResNet] was created
***Defined hyperparameters:***
"arch":                {'name': 'resnet50', 'args': {'num_channels': 1, 'num_classes': 3}}
"callbacks":           ['litprogressbar', 'modelcheckpoints', 'metriccallback']
"class_weights":       None
"hp_metrics":          ['loss/train', 'loss/val', 'accuracy/val', 'auc/val', 'specificity/val', 'sensitivity/val']
"kfold":               {'enable': False, 'folds': 10}
"loss":                CrossEntropyLoss
"loss_weight_balance": None
"optimizer":           {'type': 'Adam', 'args': {'lr': 0.0002, 'weight_decay': 1e-05, 'amsgrad': True}}
"roi_enabled":         False
"roi_hparams":         {'enable': True, 'output_shape': [40, 40, 40], 'boundary_boxes': [[17, 18, 79, 94, 2, 76]]}
Dataset sizes - Training: 421 Validation: 47


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


In [4]:
roi_model

Model(
  (model): ResNet(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv3d(64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (bn3): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsampl

In [ ]:
(256,256,256) -> (2,2,2)

(40,40,40) -> (1,1,1) -> (2,2,2)

(79,95,79) -> (1,1,1)


In [5]:
roi_trainer.fit(roi_model, datamodule=roi_dataset)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 46.2 M
---------------------------------
46.2 M    Trainable params
0         Non-trainable params
46.2 M    Total params
184.645   Total estimated model params size (MB)
/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/stud/f/fremar16/miniconda3/envs/Master-thesis/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)




Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  2887.7         	|  100 %          	|
------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  14.434         	|200            	|  2886.8         	|  99.97          	|
on_validation_end                  	|  8.2179         	|201            	|  1651.8         	|  57.201         	|
run_training_batch                 	|  0.056538       	|14200          	|  802.83         	|  27.802         	|
optimizer_step_and_closure_0       	|  0.052216       	|14200          	|  741.47         	|  25.677         	|
training_step_and_backward         	|  0.

1

In [6]:
roi_trainer.predict()

MisconfigurationException: Dataloader not found for `Trainer.predict`